In [1]:
import torch
from utils import *
from collections import defaultdict
import matplotlib.pyplot as plt
import time

from models.rendering import *
from models.nerf import *

import metrics

from datasets.DVRDataset import DVRDataset
from datasets import dataset_dict
from datasets.llff import *

torch.backends.cudnn.benchmark = True

img_wh = (160, 120)

# dataset = dataset_dict['llff'] \
#           ('/home/ubuntu/data/nerf_example_data/nerf_llff_data/fern/', 'test_train', spheric_poses=False,
#            img_wh=img_wh)

# dataset = dataset_dict['llff_nocs'] \
#           ('data/scene_1',
#            img_wh=img_wh)
# dataset = dataset_dict['objectron_multi']

# print("dataset", dataset)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
"""Methods for drawing a bounding box on an image."""
import cv2
import numpy as np

import objectron.dataset.box as Box

_LINE_TICKNESS = 10
_POINT_RADIUS = 10
_COLORS = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (128, 128, 0),
    (128, 0, 128),
    (0, 128, 128),
    (255, 255, 255),
    (0, 0, 0),
    (255, 0, 255),
]

def draw_annotation_on_image(image,
                             object_annotations,
                             num_keypoints):
  """Draw annotation on the image."""
  # The object annotation is a list of 3x1 keypoints for all the annotated
  # objects. The objects can have a varying number of keypoints. First we split
  # the list according to the number of keypoints for each object. This
  # also leaves an empty array at the end of the list.
  keypoints = np.split(object_annotations, np.array(np.cumsum(num_keypoints)))
  keypoints = [points.reshape(-1, 3) for points in keypoints]
  h, w, _ = image.shape
  num_objects = len(num_keypoints)
  # The keypoints are [x, y, d] where `x` and `y` are normalized (`uv`-system)\
  # and `d` is the metric distance from the center of the camera. Convert them
  # keypoint's `xy` value to pixel.
  keypoints = [
      np.multiply(keypoint, np.asarray([w, h, 1.], np.float32)).astype(int)
      for keypoint in keypoints
  ]

  def draw_face(object_id, face, color):
    start = keypoints[object_id][face[0], :]
    end = keypoints[object_id][face[2], :]
    cv2.line(image, (start[0], start[1]), (end[0], end[1]), color,
             _LINE_TICKNESS)
    start = keypoints[object_id][face[1], :]
    end = keypoints[object_id][face[3], :]
    cv2.line(image, (start[0], start[1]), (end[0], end[1]), color,
             _LINE_TICKNESS)

  for object_id in range(num_objects):
    num_keypoint = num_keypoints[object_id]
    edges = Box.EDGES
    hidden = [False] * Box.NUM_KEYPOINTS
    draw_face(object_id, Box.FACES[Box.FRONT_FACE_ID], _COLORS[7])
    draw_face(object_id, Box.FACES[Box.TOP_FACE_ID], _COLORS[8])

    for kp_id in range(num_keypoint):
      kp_pixel = keypoints[object_id][kp_id, :]
      # If a keypoint is hidden (e.g. a subset of a larger skeleton family) do
      # not visualize it.
      if not hidden[kp_id]:
        cv2.circle(image, (kp_pixel[0], kp_pixel[1]), _POINT_RADIUS,
                   _COLORS[object_id % len(_COLORS)], -1)

    for edge in edges:
      # This if statement is for backward compatibility, where we might later
      # add more edges/keypoints to the skeletons.
      if edge[0] < num_keypoint and edge[1] < num_keypoint:
        start_kp = keypoints[object_id][edge[0], :]
        end_kp = keypoints[object_id][edge[1], :]
        if not hidden[edge[0]] and not hidden[edge[1]]:
          cv2.line(image, (start_kp[0], start_kp[1]), (end_kp[0], end_kp[1]),
                   _COLORS[object_id % len(_COLORS)], _LINE_TICKNESS)
  return image

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [3]:
flags={}
flags["list_prefix"] = "new_"
flags["max_imgs"] = 49
flags["sub_format"] = "dtu"
flags["scale_focal"] = False
flags["z_near"] = 0.1
flags["z_far"] = 5.0
# img_wh = (160, 120)
from torch.utils.data import DataLoader
from datasets.DVRDataset import DVRDataset
from datasets.objectron_multi_all import ObjectronMultiDataset
# kwargs = {'root_dir': '/home/zubair/cup',
#           'img_wh': tuple(img_wh)}
# # kwargs['spheric_poses'] = False
# # kwargs['val_num'] = 2
# datadir = '/home/zubair/Downloads/Compressed/dtu_dataset/rs_dtu_4'

datadir = '/home/zubair/camera'

#train_dataset = DVRDataset(datadir, stage="train", **flags)
# train_dataset = DVRDataset(datadir, stage="train")
train_dataset = ObjectronMultiDataset(datadir, stage="train")

dataloader =  DataLoader(train_dataset,
                  shuffle=False,
                  num_workers=1,
                  batch_size=1,
                  pin_memory=True)

print("len train dataset", len(train_dataset))

# data_time = AverageMeter('Data', ':6.3f')
# progress = ProgressMeter(
#     len(dataloader),
#     [data_time],
#     prefix="Epoch: [{}]".format(0))

len train dataset 14


In [4]:
import torch
print_freq = 2
import time
start_time = time.time()
for i, data in enumerate(dataloader):
    print("time", time.time() - start_time)
    print("len imgs", len(data["images"]))
    for k,v in data.items():
        
        if isinstance(v, list):
            print(k, len(v))
        if torch.is_tensor(v):
            print(k, v.shape)
#         else:
#             print(k,v)

#     if i % args.print_freq == 0:
#         progress.display(i)
    print("===============\n")

end_time_objectron 0.11925029754638672
image time 0.0036764144897460938

per_frame_time 0.14238262176513672
end_time_objectron 0.06844830513000488
image time 0.002290964126586914

per_frame_time 0.08585929870605469
end_time_objectron 0.08452320098876953
image time 0.002328634262084961

per_frame_time 0.10388350486755371
end_time_objectron 0.06869888305664062
image time 0.0023741722106933594

per_frame_time 0.08252286911010742
end_time_objectron 0.07126665115356445
image time 0.002592802047729492

per_frame_time 0.0899055004119873
end_time_objectron 0.07185769081115723
image time 0.0030574798583984375

per_frame_time 0.08910465240478516
end_time_objectron 0.07069206237792969
image time 0.002321004867553711

per_frame_time 0.08728981018066406
end_time_objectron 0.07195711135864258
image time 0.0025162696838378906

per_frame_time 0.08738875389099121
end_time_objectron 0.08341002464294434
image time 0.003467082977294922

per_frame_time 0.10132551193237305
end_time_objectron 0.0755500793457

end_time_objectron 0.06986355781555176
image time 0.002969503402709961

per_frame_time 0.08867263793945312
end_time_objectron 0.06942868232727051
image time 0.0034189224243164062

per_frame_time 0.08908700942993164
end_time_objectron 0.07354974746704102
image time 0.002384662628173828

per_frame_time 0.09146547317504883
end_time_objectron 0.07167363166809082
image time 0.003630399703979492

per_frame_time 0.09217524528503418
end_time_objectron 0.09928631782531738
image time 0.003921985626220703

per_frame_time 0.11669254302978516
end_time_objectron 0.06852006912231445
image time 0.0022649765014648438

per_frame_time 0.08510780334472656
end_time_objectron 0.10879659652709961
image time 0.002282857894897461

per_frame_time 0.12589454650878906
end_time_objectron 0.09467911720275879
image time 0.0036373138427734375

per_frame_time 0.11012864112854004
end_time_objectron 0.07129216194152832
image time 0.0022606849670410156

per_frame_time 0.08576750755310059
end_time_objectron 0.080399751663

end_time_objectron 0.07236766815185547
image time 0.0026009082794189453

per_frame_time 0.08786702156066895
end_time_objectron 0.06903648376464844
image time 0.0023572444915771484

per_frame_time 0.08491802215576172
end_time_objectron 0.07196426391601562
image time 0.0036652088165283203

per_frame_time 0.08966350555419922
end_time_objectron 0.06959795951843262
image time 0.002317667007446289

per_frame_time 0.0881040096282959
end_time_objectron 0.06831526756286621
image time 0.003674030303955078

per_frame_time 0.0854032039642334
end_time_objectron 0.0718088150024414
image time 0.0036606788635253906

per_frame_time 0.08795714378356934
end_time_objectron 0.06997013092041016
image time 0.0022933483123779297

per_frame_time 0.08732247352600098
end_time_objectron 0.07589030265808105
image time 0.0023279190063476562

per_frame_time 0.09102702140808105
end_time_objectron 0.11060070991516113
image time 0.0019485950469970703

per_frame_time 0.12331056594848633
end_time_objectron 0.060049533843

end_time_objectron 0.049684762954711914
image time 0.0018131732940673828

per_frame_time 0.06182122230529785
end_time_objectron 0.04018855094909668
image time 0.002000570297241211

per_frame_time 0.05053257942199707
end_time_objectron 0.038983821868896484
image time 0.0018401145935058594

per_frame_time 0.05218648910522461
end_time_objectron 0.07072806358337402
image time 0.0027747154235839844

per_frame_time 0.08419632911682129
end_time_objectron 0.04618120193481445
image time 0.0026624202728271484

per_frame_time 0.06004738807678223
end_time_objectron 0.042621612548828125
image time 0.0019958019256591797

per_frame_time 0.05345487594604492
end_time_objectron 0.03807663917541504
image time 0.0028371810913085938

per_frame_time 0.052193641662597656
end_time_objectron 0.04953885078430176
image time 0.0028007030487060547

per_frame_time 0.06283712387084961
end_time_objectron 0.07125520706176758
image time 0.0028333663940429688

per_frame_time 0.08528876304626465
end_time_objectron 0.0391

KeyboardInterrupt: 

In [ ]:
# import torch
# import numpy as np
# a = torch.tensor([[0.6, 0.0, 0.0, 0.0],
# [0.0, 0.4, 0.0, 0.0],
# [0.0, 0.0, 1.2, 0.0],
# [0.0, 0.0, 0.0,-0.4]])
# b = torch.nonzero(a)

# print(b.shape)
# c = np.nonzero(a.numpy())
# c = np.transpose(np.asarray(c))
# print(np.transpose(np.asarray(c)).shape)
# print(np.nonzero(a.numpy()))

# print(np.equal(b.numpy(), c))

print(int(1267042/160000))

print(1195042/160000)

print(int(1195042/160000))
print(1267042%160000)

print(.9190125*160000)

In [ ]:
print("len train dataset", len(train_dataset))
count = 0
for i, data in enumerate(dataloader):
    print("i", i)
    print("================\n\n\n")
    if "new_img_wh" in data:
        W,H = data["new_img_wh"]
    for k,v in data.items():
        if k =="GT RTs" or k =="new_img_wh":
            continue
        print("k, v", k, v.shape)
        print("==============\n")
        if k =='rays':
            print(v.shape)
        if k == 'rgbs':
            print("rgbs",v.shape)
                
            rgb_array = v.squeeze(0)
            rgb = v.squeeze(0).view(H,W,3).numpy()
            plt.imshow(rgb)
            plt.show()
        if k=='instance_ids':
            print("v", v)
        if k == 'instance_mask':
            print("instance mask.shape", v.shape)
            print(v.view(-1).view(-1).shape)
            instance_mask_array = v.view(-1)
            instance_mask = v.view(-1).view(H,W).numpy()
            plt.imshow(instance_mask)
            plt.show()
#             cropped_mask = torch.tensor(cropped_mask).view(-1).numpy()
#             print("cropped_mask",cropped_mask.shape)
#             masked_rgb = np.ones(cropped_rgb.shape)
#             masked_rgb[cropped_mask] = cropped_rgb[cropped_mask]
#             print(masked_rgb.shape, "masked_rgb")
#             masked_rgb = masked_rgb.reshape(120,160,3)
#             print(masked_rgb.shape)
#             plt.imshow(masked_rgb)
#             plt.show()
    print("============================\n\n\n\n")

            
            
#         if k =='instance_ids':
#             print(v)

    count+=1
print(count)
#     print("data_rgb", data['rgbs'].shape)

In [ ]:
models = {'coarse': nerf_coarse, 'fine': nerf_fine}
embeddings = {'xyz': embedding_xyz, 'dir': embedding_dir}

N_samples = 64
N_importance = 64
use_disp = False
chunk = 1024*32*4

@torch.no_grad()
def f(rays):
    """Do batched inference on rays using chunk."""
    B = rays.shape[0]
    results = defaultdict(list)
    for i in range(0, B, chunk):
        rendered_ray_chunks = \
            render_rays(models,
                        embeddings,
                        rays[i:i+chunk],
                        N_samples,
                        use_disp,
                        0,
                        0,
                        N_importance,
                        chunk,
                        dataset.white_back,
                        test_time=True)

        for k, v in rendered_ray_chunks.items():
            results[k] += [v]

    for k, v in results.items():
        results[k] = torch.cat(v, 0)
    return results

In [ ]:
sample = dataset[0]
rays = sample['rays'].cuda()

t = time.time()
results = f(rays)
torch.cuda.synchronize()
print(time.time()-t)

In [ ]:
img_gt = sample['rgbs'].view(img_wh[1], img_wh[0], 3)
img_pred = results['rgb_fine'].view(img_wh[1], img_wh[0], 3).cpu().numpy()
alpha_pred = results['opacity_fine'].view(img_wh[1], img_wh[0]).cpu().numpy()
depth_pred = results['depth_fine'].view(img_wh[1], img_wh[0])

print('PSNR', metrics.psnr(img_gt, img_pred).item())

plt.subplots(figsize=(15, 8))
plt.tight_layout()
plt.subplot(221)
plt.title('GT')
plt.imshow(img_gt)
plt.subplot(222)
plt.title('pred')
plt.imshow(img_pred)
plt.subplot(223)
plt.title('depth')
plt.imshow(visualize_depth(depth_pred).permute(1,2,0))
plt.show()

In [ ]:
import numpy as np
def ralign(X,Y):
    m, n = X.shape

    mx = X.mean(1)
    my = Y.mean(1)
    Xc =  X - np.tile(mx, (n, 1)).T
    Yc =  Y - np.tile(my, (n, 1)).T

    sx = np.mean(np.sum(Xc*Xc, 0))
    sy = np.mean(np.sum(Yc*Yc, 0))

    Sxy = np.dot(Yc, Xc.T) / n

    U,D,V = np.linalg.svd(Sxy,full_matrices=True,compute_uv=True)
    V=V.T.copy()
    r = np.linalg.matrix_rank(Sxy)
    d = np.linalg.det(Sxy)
    S = np.eye(m)
    if r > (m - 1):
        if ( np.det(Sxy) < 0 ):
            S[m, m] = -1;
        elif (r == m - 1):
            if (np.det(U) * np.det(V) < 0):
                S[m, m] = -1  
        else:
            R = np.eye(2)
            c = 1
            t = np.zeros(2)
            return R,c,t

    R = np.dot( np.dot(U, S ), V.T)

    c = np.trace(np.dot(np.diag(D), S)) / sx
    t = my - c * np.dot(R, mx)

    return R,c,t

# Run an example test
# We have 3 points in 3D. Every point is a column vector of this matrix A
# A=np.array([[0.57215 ,  0.37512 ,  0.37551] ,[0.23318 ,  0.86846 ,  0.98642],[ 0.79969 ,  0.96778 ,  0.27493]])
# # Deep copy A to get B
# B=A.copy()
# and sum a translation on z axis (3rd row) of 10 units
A = np.ones((10,3))
B=A.copy()
B[2,:]=B[2,:]+10



print(A.shape, B.shape)

# Reconstruct the transformation with ralign.ralign
R, c, t = ralign(A,B)
print (R,c,t)

In [ ]:
import numpy as np

a = np.arange(1,21,1)

print(a)

In [ ]:
import numpy as np

val_instances = [30, 60, 90, 120, 145]
a = np.random.choice([i for i in range(0,150) if i not in val_instances])
print(a)

numbers = range(0,150)
train_indices = [n for n in numbers if n not in val_instances]
print(train_indices, len(train_indices))
import random
random.shuffle(train_indices)
print(train_indices)
print(np.random.shuffle(np.array(train_indices)))


In [ ]:
sel_indices = np.random.choice(50, 50, replace=False)

In [ ]:
print(sel_indices)

In [ ]:
import torch
def unproj_map(width, height, f, c=None, device="cpu"):
    """
    Get camera unprojection map for given image size.
    [y,x] of output tensor will contain unit vector of camera ray of that pixel.
    :param width image width
    :param height image height
    :param f focal length, either a number or tensor [fx, fy]
    :param c principal point, optional, either None or tensor [fx, fy]
    if not specified uses center of image
    :return unproj map (height, width, 3)
    """
    if c is None:
        c = [width * 0.5, height * 0.5]
    else:
        c = c.squeeze()
    if isinstance(f, float):
        f = [f, f]
    elif len(f.shape) == 0:
        f = f[None].expand(2)
    elif len(f.shape) == 1:
        f = f.expand(2)
    Y, X = torch.meshgrid(
        torch.arange(height, dtype=torch.float32) - float(c[1]),
        torch.arange(width, dtype=torch.float32) - float(c[0]),
    )
    X = X.to(device=device) / float(f[0])
    Y = Y.to(device=device) / float(f[1])
    Z = torch.ones_like(X)
    unproj = torch.stack((X, -Y, -Z), dim=-1)
    unproj /= torch.norm(unproj, dim=-1).unsqueeze(-1)
    return unproj

all_focal = torch.FloatTensor([[123.0807, 123.0807],             
        [121.7475, 121.7475],                                            
        [121.6742, 121.6742],                    
        [119.6573, 119.6573],
        [123.4082, 123.4082],                                           
        [123.5046, 123.5046],                                     
        [123.1602, 123.1602],                                             
        [125.9188, 125.9188]])
print(all_focal.shape)
focal = torch.FloatTensor([123.0807, 123.0807])
all_unproj = []
for focal in all_focal:
    print(focal)
    unproj = unproj_map(160, 120, focal)
    all_unproj.append(unproj.unsqueeze(0))
    print(unproj.shape)
unproj = torch.cat(all_unproj, dim=0)
print(unproj.shape)

In [ ]:
print(unproj.shape)